## <center> Import Pandas for EDA <center>

In [1]:
import pandas as pd

## <center> Import check_in data from Yelp dataset <center>

In [2]:
file = '/Users/Lisa/Downloads/yelp_dataset/yelp_academic_dataset_checkin.json'

In [3]:
checkins = pd.read_json(file, lines=True)

In [4]:
checkins

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."
...,...,...
175182,zzvlwkcNR1CCqOPXwuvz2A,"2017-05-06 20:05:15, 2017-05-12 22:37:03, 2017..."
175183,zzwaS0xn1MVEPEf0hNLjew,"2010-02-16 02:09:56, 2010-07-05 05:40:48, 2010..."
175184,zzwhN7x37nyjP0ZM8oiHmw,"2016-03-06 13:27:02, 2016-03-09 00:41:53, 2016..."
175185,zzwicjPC9g246MK2M1ZFBA,"2012-09-22 00:26:15, 2012-09-23 20:12:00, 2012..."


### <center> Since the date column is a long string I'll need to change it to a list in order to leverage date/time (?) <center>

In [5]:
checkins['date'] = checkins['date'].apply(lambda x: x.split(","))

In [6]:
a = checkins.iloc[0,1]

In [7]:
type(a)

list

In [8]:
checkins

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"[2016-04-26 19:49:16, 2016-08-30 18:36:57, 2..."
1,--6MefnULPED_I942VcFNA,"[2011-06-04 18:22:23, 2011-07-23 23:51:33, 2..."
2,--7zmmkVg-IMGaXbuVd0SQ,"[2014-12-29 19:25:50, 2015-01-17 01:49:14, 2..."
3,--8LPVSo5i0Oo61X01sV9A,[2016-07-08 16:43:30]
4,--9QQLMTbFzLJ_oT-ON3Xw,"[2010-06-26 17:39:07, 2010-08-01 20:06:21, 2..."
...,...,...
175182,zzvlwkcNR1CCqOPXwuvz2A,"[2017-05-06 20:05:15, 2017-05-12 22:37:03, 2..."
175183,zzwaS0xn1MVEPEf0hNLjew,"[2010-02-16 02:09:56, 2010-07-05 05:40:48, 2..."
175184,zzwhN7x37nyjP0ZM8oiHmw,"[2016-03-06 13:27:02, 2016-03-09 00:41:53, 2..."
175185,zzwicjPC9g246MK2M1ZFBA,"[2012-09-22 00:26:15, 2012-09-23 20:12:00, 2..."


## <center> Leverage dataframe.explode('column_name') to break the list of check-in's down <center>

In [9]:
checkins.explode('date')

,business_id,date
0,--1UhMGODdWsrMastO9DZw,2016-04-26 19:49:16
0,--1UhMGODdWsrMastO9DZw,2016-08-30 18:36:57
0,--1UhMGODdWsrMastO9DZw,2016-10-15 02:45:18
0,--1UhMGODdWsrMastO9DZw,2016-11-18 01:54:50
0,--1UhMGODdWsrMastO9DZw,2017-04-20 18:39:06
...,...,...
175186,zzzaIBwimxVej4tY6qFOUQ,2014-07-04 21:28:29
175186,zzzaIBwimxVej4tY6qFOUQ,2014-07-14 01:02:55
175186,zzzaIBwimxVej4tY6qFOUQ,2014-07-14 21:26:59
175186,zzzaIBwimxVej4tY6qFOUQ,2014-07-17 01:06:02


## <center> Expand the 'date' column to see what the data looks like <center>

In [10]:
test_date = checkins.iloc[0,1]
test_date

['2016-04-26 19:49:16',
 ' 2016-08-30 18:36:57',
 ' 2016-10-15 02:45:18',
 ' 2016-11-18 01:54:50',
 ' 2017-04-20 18:39:06',
 ' 2017-05-03 17:58:02',
 ' 2019-03-19 22:04:48']

## <center> From my EDA is seems like each business has a unique business_id which gives a breakdown of the dates/times people checked in to these businesses <center>

## <center> Import Business data from Yelp dataset <center>

In [11]:
file2 = '/Users/Lisa/Downloads/yelp_dataset/yelp_academic_dataset_business.json'

In [12]:
businesses = pd.read_json(file2, lines=True)
businesses

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,9750 Weston Road,Vaughan,ON,L4H 2P2,43.838555,-79.559823,4.0,5,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Japanese, Sushi Bars, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,3240 Wilkinson Blvd,Charlotte,NC,28208,35.226307,-80.893539,2.0,13,0,"{'RestaurantsPriceRange2': '1', 'BusinessParki...","Department Stores, Food, Mobile Phones, Fashio...",None
209390,SYa2j1boLF8DcGVOYfHPcA,Five Guys,7014-590 E Camelback Rd,Scottsdale,AZ,85251,33.504062,-111.929431,3.5,97,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","American (New), Food, Burgers, Restaurants, Fa...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
209391,RSSIsgO00OuWQTRoITacpA,Indian Trail Dog Training,,Indian Trail,NC,28079,35.101965,-80.599385,5.0,3,1,None,"Pet Services, Pet Training, Pets","{'Monday': '0:0-0:0', 'Tuesday': '7:0-19:0', '..."


### <center> Clean up businesses dataframe and elimnate unnecessary columns since its not data I need to focus on <center>

In [13]:
businesses = businesses.drop(['hours', 'categories', 'attributes', 'longitude', 'latitude', 'stars', 'is_open', 'address', 'postal_code'], axis=1)

In [14]:
businesses

,business_id,name,city,state,review_count
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,Cornelius,NC,36
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,4
2,XNoUzKckATkOD1hP6vghZg,Felinus,Montreal,QC,5
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,North Las Vegas,NV,3
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,26
...,...,...,...,...,...
209388,9Q0fPWAjUweoFDk0kafuzQ,Nishi Sushi,Vaughan,ON,5
209389,PotiAhtDMYMNsJCzV0oLAw,Walmart,Charlotte,NC,13
209390,SYa2j1boLF8DcGVOYfHPcA,Five Guys,Scottsdale,AZ,97
209391,RSSIsgO00OuWQTRoITacpA,Indian Trail Dog Training,Indian Trail,NC,3


In [15]:
businesses.loc[businesses['city'] == 'Austin']

,business_id,name,city,state,review_count


In [16]:
businesses.loc[businesses['state'] == 'TX']

,business_id,name,city,state,review_count
5210,wr6z01f4jcGQkRQMSw7nLg,Pizza Hut,Dallas,TX,10
79162,2SmVDHidtlfAF9bXBn4XwQ,Pizza Hut,Charlotte,TX,16
104542,7j5yndEexyQn9cpVXLIIpw,Texas de Brazil,Dallas,TX,987
114158,2H9lfg6In0Z5bFi0hbWOIg,Omaha Steakhouse,Phoenix,TX,65
123992,zZ3XzaG82sL4OU90Mpp60w,Liberty Choice Insurance,Frisco,TX,7
185411,qLWav98wzcVvjYQq4GtT7A,FedEx Office Print & Ship Center,Lufkin,TX,21


## <center> Idnetify a State with enough data points to work with (since there's a lot of entries) <center>

In [17]:
businesses.groupby('state').count()

,business_id,name,city,review_count
state,,,,
AB,8682,8682,8682,8682
AK,1,1,1,1
AL,3,3,3,3
AR,1,1,1,1
AZ,60803,60803,60803,60803
BC,2,2,2,2
CA,23,23,23,23
CO,2,2,2,2
CT,2,2,2,2


In [18]:
businesses.loc[businesses['state'] == 'AZ']

,business_id,name,city,state,review_count
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD",Scottsdale,AZ,4
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,Mesa,AZ,26
5,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,Gilbert,AZ,38
7,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,Mesa,AZ,18
13,JjJs3o60uQCfctDjs45cmA,Convertabath,Chandler,AZ,10
...,...,...,...,...,...
209376,cmsAFAymfVoBitSF8R6GKw,Yao Fine Chinese Cuisine,Gilbert,AZ,230
209378,TLEXFQf9UT4PV5QJrc3QPw,PC Pool Care,Scottsdale,AZ,6
209380,YZeUH6zYS0dq5QHLYZhUnQ,Hooters,Phoenix,AZ,106
209384,5d6Z-FjQ3g7TZxIs9mJ9kw,Sunbelt Rentals,Tempe,AZ,5
